<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/vitaldb_tableone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#테이블 생성 예제
본 예제에서는 vitaldb 데이터셋으로부터 사망 여부에 따른 군간 테이블을 그려본다.

In [1]:
!pip install tableone

In [5]:
import scipy.stats as stat
from tableone import TableOne
import pandas as pd

# 데이터를 읽음
df = pd.read_csv('https://api.vitaldb.net/cases')

# 통계에 사용하지 않을 변수들을 제외
df.drop(columns=['caseid', 'subjectid', 'dx', 'opname', 'casestart', 'iv1', 'iv2', 'aline1', 'aline2', 'cline1', 'cline2', 'filesize', 'lmasize'], inplace=True)

# 다음 변수들을 시간, 날짜 단위로 변경
df.loc[:, ['caseend', 'anestart', 'aneend', 'opstart', 'opend', 'casedur', 'opdur', 'anedur']] /= 3600
df.loc[:, ['adm', 'dis']] /= 3600 * 24

# 카테고리 변수들을 지정
categoricals = [col for col in df.columns if df[col].dtype == 'object' or df[col].dtype == 'str' or len(df[col].unique()) < 7]

# 비정규 분포인 변수들을 구함
nonnormals = [col for col in df.columns if col not in categoricals and stat.normaltest(df[col], nan_policy='omit')[1] < 0.05]

# 테이블을 생성
tbl = TableOne(df, columns=list(df.columns), categorical=categoricals, nonnormal=nonnormals, groupby='death_inhosp', missing=False, pval=True, htest_name=True)
pd.set_option('display.max_rows', 500)
tbl

Grouped by death_inhosp                                                                                                
                                                                                                                                             Overall                     0                      1 P-Value                                       Test
n                                                                                                                                               6388                  6331                     57                                                   
caseend, median [Q1,Q3]                                                                                                                2.8 [1.7,4.2]         2.8 [1.7,4.2]          2.7 [1.9,4.5]   0.376                             Kruskal-Wallis
anestart, median [Q1,Q3]                                                                                                            -0.1 [-0.3,-0.1]      -0.1 [-0.3,-0.1]       -0.1 [-0.3,-0.1]   0.517                             Kruskal-Wallis
aneend, median [Q1,Q3]                                                                                                                 2.7 [1.7,4.2]         2.7 [1.7,4.2]          2.7 [1.7,4.6]   0.232                             Kruskal-Wallis
opstart, median [Q1,Q3]                                                                                                               0.4 [-0.1,0.7]        0.4 [-0.1,0.7]         0.4 [-0.1,0.7]   0.796                             Kruskal-Wallis
opend, median [Q1,Q3]                                                                                                                  2.5 [1.5,4.0]         2.5 [1.5,4.0]          2.5 [1.6,4.4]   0.246                             Kruskal-Wallis
adm, median [Q1,Q3]                                                                                                                 -2.3 [-2.6,-1.5]      -2.3 [-2.6,-1.5]      -2.8 [-16.8,-2.3]  <0.001                             Kruskal-Wallis
dis, median [Q1,Q3]                                                                                                                    4.6 [2.4,7.5]         4.5 [2.4,7.5]        13.5 [5.0,30.4]  <0.001                             Kruskal-Wallis
devices, median [Q1,Q3]                                                                                                                5.0 [4.0,5.0]         5.0 [4.0,5.0]          5.0 [4.0,5.0]   0.307                             Kruskal-Wallis
tracks, median [Q1,Q3]                                                                                                              78.0 [73.0,83.0]      78.0 [74.0,83.0]       78.0 [72.0,83.0]   0.505                             Kruskal-Wallis
casedur, median [Q1,Q3]                                                                                                                0.0 [0.0,0.1]         0.0 [0.0,0.1]          0.0 [0.0,0.1]   0.450                             Kruskal-Wallis
age, median [Q1,Q3]                                                                                                                 59.0 [48.0,68.0]      59.0 [48.0,68.0]       61.0 [47.0,73.0]   0.452                             Kruskal-Wallis
sex, n (%)                          F                                                                                                    3145 (49.2)           3125 (49.4)              20 (35.1)   0.044                                Chi-squared
                                    M                                                                                                    3243 (50.8)           3206 (50.6)              37 (64.9)                                                   
height, median [Q1,Q3]                                                                                                           162.2 [156.1,168.7]   162.1 [156.1,168.7]    165.4 [158.2,171.0]   0.392    